In [0]:
!pip install langchain databricks-langchain --quiet

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import os
from databricks_langchain import ChatDatabricks
from langchain.tools import tool

In [0]:
os.environ["DATABRICKS_HOST"] = "{Put your Databricks Host(URL) here}
os.environ["DATABRICKS_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

In [0]:
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

# Asking generic questions

In [0]:
llm.invoke("Who is the Prime Minister of Inida from 2014?")

AIMessage(content='The Prime Minister of India from 2014 is Narendra Modi. He has been serving as the Prime Minister of India since May 26, 2014, and was re-elected for a second term in 2019.', additional_kwargs={}, response_metadata={'usage': {'prompt_tokens': 23, 'completion_tokens': 48, 'total_tokens': 71}, 'prompt_tokens': 23, 'completion_tokens': 48, 'total_tokens': 71, 'model': 'meta-llama-4-maverick-040225', 'model_name': 'meta-llama-4-maverick-040225', 'finish_reason': 'stop'}, id='lc_run--bc42bd26-6e00-455e-94c1-822d27cec8e2-0')

# Asking questions which require real-time data

In [0]:
llm.invoke("What is the current time?")

AIMessage(content='I\'m not capable of accessing real-time information, so I can\'t provide you with the current time. However, I can suggest a few alternatives to find out the current time:\n\n1. Check your device: You can check the time on your phone, watch, or computer.\n2. Use a search engine: You can type "current time" in a search engine like Google, and it will display the current time based on your location or a specific time zone if you specify it.\n3. Look at a clock: If you\'re near a clock, you can simply look at it to know the current time.\n\nIf you need to know the time in a specific time zone or city, feel free to let me know, and I can try to help you with that!', additional_kwargs={}, response_metadata={'usage': {'prompt_tokens': 16, 'completion_tokens': 154, 'total_tokens': 170}, 'prompt_tokens': 16, 'completion_tokens': 154, 'total_tokens': 170, 'model': 'meta-llama-4-maverick-040225', 'model_name': 'meta-llama-4-maverick-040225', 'finish_reason': 'stop'}, id='lc_ru

# Tool creation

In [0]:
from langchain.tools import tool

class ToolsList:
    @tool
    def get_current_time() -> str:
        """
        Returns the current timestamp.

        This tool fetches the system's current local date and time. 
        It is useful for tasks where the LLM needs to reference the actual 
        current time instead of reasoning about it contextually.

        Returns:
            str: The current local timestamp in the format YYYY-MM-DD HH:MM:SS.
        """
        from datetime import datetime
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    @tool
    def custom_calculator(expression: str) -> str:
        """
        Evaluates a mathematical expression provided as a string.

        This tool safely parses and evaluates arithmetic expressions such as 
        addition, subtraction, multiplication, division, and parentheses-based 
        grouping. It filters out invalid characters before evaluation to prevent 
        code injection or unsafe execution.

        Args:
            expression (str): A mathematical expression, e.g., "2 + 3 * (5 - 1)".

        Returns:
            str: The result of the evaluated expression as a string.
                Returns an error message if the expression is invalid or 
                leads to a runtime error (e.g., division by zero).

        Example:
            >>> custom_calculator("10 / 2 + 3")
            '8.0'
        """
        import re

        # Remove any non-digit or non-operator characters
        expression = re.sub(r'[^0-9+\-*/().]', '', expression)
        try:
            result = eval(expression)
            return str(result)
        except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
            return "Error: Invalid expression"


# Tool binding

In [0]:
tools_list = [ToolsList.get_current_time, ToolsList.custom_calculator]
llm_with_tools = llm.bind_tools(tools_list)

# Tool calling

In [0]:
ai_msg = llm_with_tools.invoke("Hello world!")
ai_msg

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'usage': {'prompt_tokens': 693, 'completion_tokens': 10, 'total_tokens': 703}, 'prompt_tokens': 693, 'completion_tokens': 10, 'total_tokens': 703, 'model': 'meta-llama-4-maverick-040225', 'model_name': 'meta-llama-4-maverick-040225', 'finish_reason': 'stop'}, id='lc_run--fadf55aa-7289-4509-8368-cfdbd3d1994a-0')

In [0]:
ai_msg = llm_with_tools.invoke("What is the Current Time?")
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7eb551ba-48ab-416f-9542-3f0bf826c225', 'function': {'arguments': '{}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'usage': {'prompt_tokens': 696, 'completion_tokens': 5, 'total_tokens': 701}, 'prompt_tokens': 696, 'completion_tokens': 5, 'total_tokens': 701, 'model': 'meta-llama-4-maverick-040225', 'model_name': 'meta-llama-4-maverick-040225', 'finish_reason': 'tool_calls'}, id='lc_run--3a961c0c-bb8f-4b0a-852f-80ccbe344b9e-0', tool_calls=[{'name': 'get_current_time', 'args': {}, 'id': 'call_7eb551ba-48ab-416f-9542-3f0bf826c225', 'type': 'tool_call'}])

# Tool execution

In [0]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_current_time": ToolsList.get_current_time, "custom_calculator": ToolsList.custom_calculator}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    print(tool_msg)

content='2025-10-23 17:13:06' name='get_current_time' tool_call_id='call_7eb551ba-48ab-416f-9542-3f0bf826c225'


# Beautification

In [0]:
# Step 1: Take a question from the user
# This simulates a real chat scenario where the user asks something
user_query = input("Enter your question here: ")

# Step 2: Import HumanMessage from LangChain core messages
# HumanMessage is used to wrap the user's input in a message object
from langchain_core.messages import HumanMessage

# Step 3: Create a list of messages to maintain conversation history
# Start with the user's question wrapped as a HumanMessage
messages = [HumanMessage(user_query)]

# Step 4: Invoke the LLM with tools using the current conversation messages
# llm_with_tools knows about the bound tools (get_current_time, custom_calculator)
# The model may decide to call a tool based on the user query
ai_msg = llm_with_tools.invoke(messages)

# Step 5: Check if the model wants to call any tools
# If tool_calls is not empty, the model has decided that a tool invocation is required
if ai_msg.tool_calls!=[]:
    messages.append(ai_msg)

    # Step 6: Iterate through any tool calls the model has requested
    # ai_msg.tool_calls contains information about which tool(s) to invoke
    # Each tool_call has a 'name' and 'args' dictionary
    for tool_call in ai_msg.tool_calls:
        
        # Step 6a: Map the tool name from the tool_call to the actual Python function
        # We lowercase the tool name to ensure matching is case-insensitive
        selected_tool = {
            "get_current_time": ToolsList.get_current_time,
            "custom_calculator": ToolsList.custom_calculator
        }[tool_call["name"].lower()]

        # Step 6b: Invoke the selected tool with the arguments provided by the model
        # The tool returns a ToolMessage containing the output of the function
        tool_msg = selected_tool.invoke(tool_call)

        # Step 6c: Append the tool's output to the messages list
        # This keeps the conversation history complete: HumanMessage -> AIMessage -> ToolMessage
        messages.append(tool_msg)

    # Step 7: Pass the updated conversation (user query + AI response + tool outputs) back to the LLM
    # This allows the model to generate a final "beautified" response incorporating:
    #   1. The original user question (HumanMessage)
    #   2. Its intermediate reasoning or AI response (AIMessage)
    #   3. Any executed tool outputs (ToolMessage)
    # The final response will be coherent and naturally formatted for the user
    print("Final Input for output generation: ")
    print(messages)
    final_response = llm_with_tools.invoke(messages)

    # Step 8: Optionally, print the final response to see the beautified output
    print(final_response)

# Case when no tool is required
# If the model did not request any tool, the AIMessage content is already the final answer
else:
    print(ai_msg.content)


Enter your question here:  What is the result of 1,984,135 * 9,343,116?

Final Input for output generation: 
[HumanMessage(content='What is the result of 1,984,135 * 9,343,116?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_c4d2e30e-57e7-41b7-97a7-1b1bb4f5b1a1', 'function': {'arguments': '{"expression": "1984135 * 9343116"}', 'name': 'custom_calculator'}, 'type': 'function'}]}, response_metadata={'usage': {'prompt_tokens': 709, 'completion_tokens': 16, 'total_tokens': 725}, 'prompt_tokens': 709, 'completion_tokens': 16, 'total_tokens': 725, 'model': 'meta-llama-4-maverick-040225', 'model_name': 'meta-llama-4-maverick-040225', 'finish_reason': 'tool_calls'}, id='lc_run--200228d6-4048-4e9e-8eb8-66b3b2309080-0', tool_calls=[{'name': 'custom_calculator', 'args': {'expression': '1984135 * 9343116'}, 'id': 'call_c4d2e30e-57e7-41b7-97a7-1b1bb4f5b1a1', 'type': 'tool_call'}]), ToolMessage(content='18538003464660', name='custom_calculator', tool_call_id='call_c4d2e30e-57e7-41b7-97a7-1b1bb4f5b1a1')]
